In [27]:
from Bio import AlignIO, SeqIO, Seq, pairwise2
from Bio.pairwise2 import format_alignment
import pandas as pd
import os
import numpy as np

In [28]:
min_length_aligned_sequence = 30 #Minimal lenght of sequence which could be an exon
extreme_homology = 0.97 #percentage of homology of sequence, treshold #I assume two faulty aligned nucleotides per 100 (98%) and one more nt because sometimes latest nt can move from end of one sequence to beginning next sequence
middle_homology = 0.9 #used for classification of internal exons
path_to_file_before_MAFFT = "/home/norbert/mrdn/euglena/kod_i_pliki/surowe_pliki_plus_minus_500"
path_to_file_after_MAFFT = '/home/norbert/mrdn/euglena/kod_i_pliki/alingment_gen_transkrypt/'

In [29]:
#MEGA ISTOTNE PYTANIE - CO WYŚWIETLAMY????
                    #CZY W KOLUMNIE SEQUENCE WYSWIETLAMY SEKWENCJE TRANKRYPTU CZY GENU?
#######################################################################################
#######################################################################################

# 1.  Z jednego pliku fasta tworyzmy 3 pliki kazdy dla innego gatunku #FUNKCJA NAPISANA - CZEKAM NA ZWOLNIENIE SIE SERWERA
# 2. robimy im maffta na zadanych parametrach #SKRYPT JEST NAPISANY
# 2.5 ZMIENIAMY troche ta funkcje by byla zaadaptowana do nowego formatu plikow. Moze nawet fasta - sprawdzmy to
# 3. Wklepujemy te pliki do naszej funkcji cutting scrap
# 4. gff tabelka i wysylamy Ani
# 5. analiza tych plikow

In [30]:
def cutting_scrap(path_to_file_before_MAFFT, path_to_file_after_MAFFT, acceptable_gap_length):
    data_frames = []
    gff_data_frames = []
    gaps_signs = "-" * acceptable_gap_length #maximum length of gaps in sequence in one exon's sequence
    
    def extracting_strands_from_alignment(alignment): #that function describe what strand is transcript strand and genomic strand
        upper_seq = alignment[0].seq
        lower_seq = alignment[1].seq
        gaps_amount_upper = upper_seq.count("-")
        gaps_ammount_lower = lower_seq.count("-")
        if gaps_amount_upper > gaps_ammount_lower:
            seq1 = upper_seq #transcript sequence
            seq2 = lower_seq #gene sequence
        else:
            seq1 = lower_seq
            seq2 = upper_seq
        return seq1, seq2

    #def setting_first_nucleotide_pair(alignment): #ta funkcja ucina nam 6 odczytow #to nam nie działa więc nie brnę dalej. Nie działa prawdopodobnie dlatego, że jest ona próba odwzorowania kodu, który spokojnie sobie działał w pętli while true
        #if "-" in alignment[:, 0]: #deleting mismatches at the first nucleotide
            #return alignment[:, 1:]
        #else:
            #return alignment
            #print(f"alignment after cut:\n{alignment}") #chekpoint


    #def setting_last_nuckleotide_pair(alignment): #to nam nie działa więc nie brnę dalej
        #if "-" in alignment[:, -1]: #deleting mismatches at the end
            #alignment = alignment[:, :-1]
            #return alignment 
        #else:
            #return alignment #print(f"alignment check:\n{alignment[:, -50:-1]}") checkpoint
                    

    def finding_gap_index(sequence, acceptable_gap_length):
        return sequence.find("-" * acceptable_gap_length) 


    def reversed_finding_gap_index(sequence, acceptable_gap_length):
        return sequence.rfind("-" * acceptable_gap_length) + acceptable_gap_length #now we have to add length of gap to index, because python starts counting from 0 nt, and we just change direction of counting.

    def setting_distance_between_first_nt_and_gap(is_it_exon):
        if is_it_exon == True:
            if first_gap_position_seq2 < 0: #when in second strand there are no gaps - it means, second strand is correctly prepared
                distance_between_first_nt_and_gap = first_gap_position_seq1
            else:
                distance_between_first_nt_and_gap = min(first_gap_position_seq1, first_gap_position_seq2)
            return distance_between_first_nt_and_gap
        elif is_it_exon == False:
            if last_gap_position_seq2 < 0: #it means, if there is no gaps in second sequention, gap position is gap position in seq1. Additional, rfind() function return "-1" if it won't find any gaps in seq
                distance_between_first_nt_and_gap = (len(seq1) - last_gap_position_seq1) -1
                #print("pierwszy", distance_between_first_nt_and_gap) #checkpoint
            else:
                distance_between_first_nt_and_gap = (len(seq1) - max(last_gap_position_seq1, last_gap_position_seq2))
            return distance_between_first_nt_and_gap
                #print("drugi", distance_between_first_nt_and_gap) #checkpoint

            
    def describing_class_of_exon(percent_of_homology, last_nt_exon_index, min_length_aligned_sequence, class_of_exon):
        if percent_of_homology >= 0.9 and last_nt_exon_index >= min_length_aligned_sequence :
            class_of_exon.append("1")
        elif percent_of_homology < 0.9 and percent_of_homology > 0 and last_nt_exon_index < min_length_aligned_sequence:
            class_of_exon.append("2")
        else:
            class_of_exon.append("3")

  
    def exons_indices(is_it_exon, max_genomic_seq, seq2, last_nt_exon_index, first_nt_position, last_nt_position):
        if is_it_exon:
            first_nt_index_of_exon = max_genomic_seq.find(str(seq2[:last_nt_exon_index]).replace("-", ""))
            if first_nt_index_of_exon == -1:
                first_nt_position.append(0)
                last_nt_position.append(0)
                print(f"That exon: {seq2[:last_nt_exon_index].replace('-', '')} comes from genomic sequence and cannot be found in file_before_MAFFT.")
            else:
                first_nt_position.append(first_nt_index_of_exon + 1)
                last_nt_position.append(first_nt_index_of_exon + len(seq2[:last_nt_exon_index]))
                


    def introns_indices(is_it_exon, max_genomic_seq, seq2, last_nt_intron_index, first_nt_position, last_nt_position):
        if is_it_exon: 
            first_nt_index_of_intron = max_genomic_seq.find(str(seq2[:last_nt_intron_index]))
            if first_nt_index_of_intron == -1:
                first_nt_position.append(0)
                last_nt_position.append(0)
            else:
                first_nt_position.append(first_nt_index_of_intron + 1)
                last_nt_position.append(first_nt_index_of_intron + len(seq2[:last_nt_intron_index]))
                

    
    def save_to_file(final_data_frame, filename):
        if os.path.isfile(filename):
            user_input = input("File already exists. Do you want overwrite? y/n: ")
            if user_input == "n":
                base_name, ext = os.path.splitext(filename)
                i = 1
                while os.path.isfile(f"{base_name}_{i}{ext}"):
                    i += 1
                filename = f"{base_name}_{i}{ext}"
                final_data_frame.to_csv(filename, sep = "\t")
                
            elif user_input == "y":
                final_data_frame.to_csv(filename, sep = "\t")
            else:
                print("Clarify your answer. Nothing has done.")
                
        else:
            final_data_frame.to_csv(filename, sep = "\t")
            
    
    for filename in os.listdir(path_to_file_after_MAFFT):
        file = os.path.join(path_to_file_after_MAFFT, filename)
        if not os.path.isfile(file):
            continue
        if file.endswith(".aln"):
            alignment = AlignIO.read(file, "clustal")
            
            #czy ten fragment trzeba zamknąć do funkcji i wyeksportować ją na początek - mówie tu o kwestiach estetycznych
            for filename_before_MAFFT in os.listdir(path_to_file_before_MAFFT):
                file_before_MAFFT = os.path.join(path_to_file_before_MAFFT, filename_before_MAFFT)
                if not os.path.isfile(file_before_MAFFT):
                    #print(f"This thingy: {str(filename_before_MAFFT)} could not be used to pair with alignment. Check it or ignore")
                    continue
                if filename_before_MAFFT.endswith(".fasta"):
                    if filename.startswith(filename_before_MAFFT[:12]):
                        max_len = 0
                        max_genomic_seq = ""
                        for record in SeqIO.parse(file_before_MAFFT, "fasta"): #getting genomic sequence - usually it is longer seq than read
                            sequence = record.seq
                            if len(sequence) > max_len:
                                max_len = len(sequence)
                                max_genomic_seq = record.seq   
                         
            while True:
                is_it_exon = True
                #alignment = setting_first_nucleotide_pair(alignment) #function that doesn't work
                first_nucleotides_pair = alignment[:, 0]#first nucleotides in both strands as variable
                if "-" in first_nucleotides_pair: #deleting mismatches at the start 
                    alignment = alignment[:, 1:]
                    #print(f"alignment after cut:\n{alignment}") #chekpoint
                    continue
    
                
                seq1, seq2 = extracting_strands_from_alignment(alignment)

                
                first_gap_position_seq1 = finding_gap_index(seq1, acceptable_gap_length)
                first_gap_position_seq2 = finding_gap_index(seq2, acceptable_gap_length)
                #print(f"first gap position in first sequence:{first_gap_position_seq1}, in 2nd sequence: {first_gap_position_seq2}") #checkpoint

                distance_between_first_nt_and_gap = setting_distance_between_first_nt_and_gap(is_it_exon)
                
                local_alignment = pairwise2.align.localxx(seq1[:distance_between_first_nt_and_gap], seq2[:distance_between_first_nt_and_gap], one_alignment_only = True) #"xx"  means no gap penalty while opening gaps or longering them and no penalties for mismatch. Just pure score of alignment to count homology
                #print("local_alignment:", local_alignment) #checkpoint
                   
                try:  #instruction what to do when seq is too short
                    local_homology_percentage = (local_alignment[0].score / distance_between_first_nt_and_gap) 
                except IndexError:
                    if len(seq1) < 100:
                        print(f"\n \n {alignment[0].id} sequence is too short, sequence has not exons or other fault. \n")
                        break
                    continue
                    

                
                if local_homology_percentage <= extreme_homology:
                    #print(f"\n Too low score of homology. \n alignment: {alignment[:, :50]} \n score: {local_alignment[0].score}, distance: {distance_between_first_nt_and_gap}, percent: {round(local_homology_percentage, 2)*100}%, \n {local_alignment}")
                    alignment = alignment[:, distance_between_first_nt_and_gap:] 
                    
                else:
                    if distance_between_first_nt_and_gap <= min_length_aligned_sequence:
                        #print(f"\n alignment is too short. \n distance between sequences {distance_between_first_nt_and_gap}, \n alignment: {alignment}, \n score: {local_alignment[0].score}, distance: {distance_between_first_nt_and_gap}, procentowo: {round(local_homology_percentage, 2)*100}%, \n {local_alignment}")
                        alignment = alignment[:, distance_between_first_nt_and_gap:]
                    else:
                        #print(f"Sequence {alignment[0].id} cut properly from LEFT side, \n distance between sequences: {distance_between_first_nt_and_gap}"), alignment: {alignment[:, :50]},
                        break
    
               
        
            
        ######################### TERAZ OD PRAWEJ DO LEWEJ #######################  

    
            
            while True:
                is_it_exon = False
                last_nucleotides_pair = alignment[:, -1]#last nucleotides in both strands as variable
                if "-" in last_nucleotides_pair: #deleting mismatches at the end
                    alignment = alignment[:, :-1]
                     #print(f"alignment check:\n{alignment[:, -50:-1]}") checkpoint
                    continue
                             
                seq1, seq2 = extracting_strands_from_alignment(alignment)

                last_gap_position_seq1 = reversed_finding_gap_index(seq1, acceptable_gap_length)
                last_gap_position_seq2 = reversed_finding_gap_index(seq2, acceptable_gap_length)
                #print("seq1:", last_gap_position_seq1, "seq2:", last_gap_position_seq2) #checkpoint
                
                distance_between_first_nt_and_gap = setting_distance_between_first_nt_and_gap(is_it_exon)
                #print("drugi", distance_between_first_nt_and_gap)
                        
                local_alignment = pairwise2.align.localxx(seq1[-distance_between_first_nt_and_gap:], seq2[-distance_between_first_nt_and_gap:], one_alignment_only = True) #"xx" means no gap penalty while opening gaps or longering them and no penalties for mismatch. Just pure score of alignment
                #print("local_alignment:", local_alignment) #checkpoint
                
                
        
                try:#instruction what to do when seq is too short or gaps were not found
                    if distance_between_first_nt_and_gap != 0:
                        local_homology_percentage = (local_alignment[0].score / distance_between_first_nt_and_gap) 
                    else:
                        break
                except IndexError:
                    if len(seq1) < 100:
                        print(f"\n \n {alignment[0].id} sequence is too short or sequence has not exons. \n")
                        break
                    continue
        
                
                if local_homology_percentage <= extreme_homology:
                     #print(f"\n Too low homology! \n alignment: {alignment[:, -50:]} \n score: {local_alignment[0].score}, \distance: {distance_between_first_nt_and_gap}, %%%: {round(local_homology_percentage, 2)*100}%, \n {local_alignment}")
                    alignment = alignment[:, :-distance_between_first_nt_and_gap] 
                    
                else:
                    if distance_between_first_nt_and_gap <= min_length_aligned_sequence:
                        #print(f"\n Alignment is too short!. \n distance: {distance_between_first_nt_and_gap}, \n alignment: {alignment}, \n score: {local_alignment[0].score}, distance: {distance_between_first_nt_and_gap}, %%%: {round(local_homology_percentage, 2)*100}%, \n {local_alignment}")
                        alignment = alignment[:, :-distance_between_first_nt_and_gap]
                    else:
                        #print(f"Sequence: {alignment[0].id} cut properly from RIGHT side, alignment: \n {alignment[:, -50:]}")
                        break

        
        
            
            nucleotides = ["a", "t", "g", "c", "A", "T", "G", "C"]
            index = 0
            
            id = []
            exon_or_intron_number = []
            is_it_intron = []
            class_of_exon = []
            percent_of_homology_for_exons = []
            length = []
            first_nt_position = []
            last_nt_position = []
            first_10_nt = []
            last_10_nt = []
            path = []
            sequence = []

            
            while True: #instruction what to do when seq is too short or gaps were not found
                try: 
                    if distance_between_first_nt_and_gap != 0:
                        local_homology_percentage = (local_alignment[0].score / distance_between_first_nt_and_gap) #that sign "[]" becouse local_alignment's type is list
                    else:
                        break
                except IndexError:
                    if len(seq1) < 100:
                        print(f"\n \n {alignment[0].id} sequence is too short or sequence has not exons. \n")
                        break
                    continue

                if len(seq1) > 1 and seq1[0] != gaps_signs:#mining data from exons
                    is_it_exon = True
                    index += 1
                    last_nt_exon_index = seq1.find(gaps_signs) 
                
                    if last_nt_exon_index == -1: #it means if it is last exon, if find() don't find any gap signs at the end
                        last_nt_exon_index = len(seq1)
                                
                    alignment_of_exon = pairwise2.align.localxx(seq1[:last_nt_exon_index], seq2[:last_nt_exon_index], one_alignment_only = True)
                        #print( "\n alignment since start to the end of exon:", alignment_of_exon[0])
                    percent_of_homology = alignment_of_exon[0].score / last_nt_exon_index
                        #print("exon analising finished")
                            
            
                    #extending lists to make bigger dictionary and then data frame
                    id.append(alignment[0].id)
                    exon_or_intron_number.append(f"E{index}")
                    is_it_intron.append(False)
                    percent_of_homology_for_exons.append(round(percent_of_homology * 100, 2))
                    length.append(last_nt_exon_index)
                    first_10_nt.append(str(seq1[:10]))
                    last_10_nt.append(str(seq2[last_nt_exon_index-10 : last_nt_exon_index])) 
                    sequence.append(str(seq2[:last_nt_exon_index]).replace("-", ""))
                    path.append(str(file))
       
                    exons_indices(is_it_exon, max_genomic_seq, seq2, last_nt_exon_index, last_nt_position, first_nt_position)
                    
                    describing_class_of_exon(percent_of_homology, last_nt_exon_index, min_length_aligned_sequence, class_of_exon)
            
                    seq1 = seq1[last_nt_exon_index:]
                    seq2 = seq2[last_nt_exon_index:]
                    #print("analising exon finished") #checkpoint
                            
                #expanding index
                if len(seq1) > 1 and seq1[0] == "-":
                    is_is_exon = False
                    for i, nukleotide in enumerate(seq1):
                        if nukleotide not in nucleotides:
                            last_nt_intron_index = i+1
                        else:
                            break

                    #extending lists to make bigger dictionary and then data frame
                    id.append(alignment[0].id)
                    exon_or_intron_number.append(f"I{index}")
                    is_it_intron.append(True)
                    class_of_exon.append(None)
                    percent_of_homology_for_exons.append(None)
                    length.append(last_nt_intron_index)
                    #first_nt_position.append(0)
                    #last_nt_position.append(0)
                    first_10_nt.append(str(seq2[:10]))
                    last_10_nt.append(str(seq2[last_nt_intron_index-10 : last_nt_intron_index]))
                    sequence.append(str(seq2[:last_nt_intron_index]).replace("-", ""))
                    path.append(str(file))

                    introns_indices(is_it_exon, max_genomic_seq, seq2, last_nt_intron_index, first_nt_position, last_nt_position)

                    seq1 = seq1[last_nt_intron_index:]
                    seq2 = seq2[last_nt_intron_index:]
                    
                else:
                     break
                        
            dictionary = {"ID":id,
                          "exon_or_intron_number":exon_or_intron_number, 
                          "is_it_intron":is_it_intron, 
                          "class_of_exon":class_of_exon, 
                          "percent_of_homology_for_exons":percent_of_homology_for_exons, 
                          "length":length,
                          "first_nt_position" : first_nt_position,
                          "last_nt_position" : last_nt_position,
                          "first_10_nt" : first_10_nt,
                          "last_10_nt" : last_10_nt, 
                          "path":path, 
                          "sequence" : sequence}
#EL|STRG.5113.1|scaffold_3161:4719-42082 manual_annotation   transcript  505 37862   1000    +   .   EL|STRG.5113.1|scaffold_3161:4719-42082 #tak ma wygladac calosc
#EL STRG.5113.1       nazwa backbone
#scaffold_3161:4719-42082 manual_annotation            źródło (np stringtie, manual)
#transcript           transcript/exon
#505        start
#37862      end
#1000       pokrycie
#+          strand (+ lub -)
#.          kropka nie wiadomo po co
#EL|STRG.5113.1|scaffold_3161:4719-42082          dłuższa nazwa sekwencji (gene_id "STRG.1"; transcript_id "STRG.1.1"; cov "52.968773"; FPKM "5.233016"; TPM "3.238127";) #jakies tam inne informacje oddzielone tym czyms ;;;;



#OG0031981.fasta:LON SL- GGOE01042269.1
#OG0031981.fasta:GRA SL- TRINITY_DN37780_c0_g2_i5
#OG0031981.fasta:HIE SL- TRINITY_DN53499_c0_g1_i4
#OG0031981.fasta:LON scaffold_16993
#OG0031981.fasta:GRA Backbone_7059
#OG0031981.fasta:HIE scaffold_10332
#OG0031981 + HIE/LON/GRA musi byc seqid
#TRINITY scaffold Backbone + numerki na końcu to
#+ - strand
#dodatkowe informacje o pokryciu a moze tez tutaj bedzie ID

  
            seqid = [] #name of the chromosome or scaffold; chromosome names can be given with or without the 'chr' prefix. Important note: the seq ID must be one used within Ensembl, i.e. a standard chromosome name or an Ensembl identifier such as a scaffold ID, without any additional content such as species or assembly. See the example GFF output below.
            source = [] #tutaj pominiemy ten krok - damy kropke #Describes the algorithm or the procedure that generated this feature. Typically Genescane or Genebank, respectively. #scaffold backbone trinity  i te numerki jakieś
            type = [] #Describes what the feature is (mRNA, domain, exon, etc.). #scaffold backbone trinity  i te numerki jakieś #intron lub exon
            start = []
            end = []
            score = [] #Typically E-values for sequence similarity and P-values for predictions. Homology there
            strand = []
            phase = [] #Indicates where the feature begins with reference to the reading frame. The phase is one of the integers 0, 1, or 2, indicating the number of bases that should be removed from the beginning of this feature to reach the first base of the next codon.. Propably 0
            attributes = [] #A semicolon-separated list of tag-value pairs, providing additional information about each feature. Some of these tags are predefined, e.g. ID, Name, Alias, Parent . You can see the full list [here](https://github.com/The-Sequence-Ontology/Specifications/blob/master/gff3.md).
            data_frame = pd.DataFrame(dictionary)

            



            
            gff_dictionary = {}

            
            gff_data_frame = pd.DataFrame(gff_dictionary)
            #print(data_frame)
        data_frames.append(data_frame)
        gff_data_frames.append(gff_data_frame)
        #print(data_frames)


    final_data_frame = pd.concat(data_frames, ignore_index=True)
    final_data_frame.index = np.arange(1, len(final_data_frame) + 1)
    print(final_data_frame)
    
    gff_final_data_frame = pd.concat(gff_data_frames, ignore_index = True)
    gff_final_data_frame.index = np.arange(1, len(gff_final_data_frame) + 1)
    print(gff_final_data_frame)
    
    #save_to_file(final_data_frame, "przyciete_introny.tsv")


cutting_scrap(path_to_file_before_MAFFT, path_to_file_after_MAFFT, 2)







 
 EL_OG0030308_t2_g2_EL_PAM1_GP153_OV05 sequence is too short, sequence has not exons or other fault. 

                                        ID exon_or_intron_number is_it_intron  \
1    EG_OG0030307_t1_g1_EG_PAM1_GP153_OV05                    E1          0.0   
2    EG_OG0030307_t1_g1_EG_PAM1_GP153_OV05                    I1          1.0   
3    EG_OG0030307_t1_g1_EG_PAM1_GP153_OV05                    E2          0.0   
4    EG_OG0030307_t1_g1_EG_PAM1_GP153_OV05                    I2          1.0   
5    EG_OG0030307_t1_g1_EG_PAM1_GP153_OV05                    E3          0.0   
..                                     ...                   ...          ...   
144  EG_OG0030309_t3_g3_EG_PAM1_GP153_OV05                    E6          0.0   
145  EG_OG0030309_t3_g3_EG_PAM1_GP153_OV05                    I6          1.0   
146  EG_OG0030309_t3_g3_EG_PAM1_GP153_OV05                    E7          0.0   
147  EG_OG0030309_t3_g3_EG_PAM1_GP153_OV05                    I7          1.0   
148